In [115]:
#%matplotlib notebook
%matplotlib inline

import json
from collections import OrderedDict
import copy
import numpy as np
import matplotlib.pyplot as plt
from _converter import SensorThings2Dict
from _plotter import plot_confusion_matrix

In [122]:
with open("ABU1.txt") as f:
    data = []
    bad = 0
    for line in f:
        try:
            features = SensorThings2Dict(line)
            data.append(list(features.values()))
        except Exception, e:
            bad+=1
print("Incomplete rows: {}".format(bad))
print("Loaded: {}".format(len(data)))

""" random split seed """
data = np.asarray(data)
#print(data[:,2:-1])
# numerics = data[:,2:-1]
# for n in numerics[data[:,-1]=='True']:
#     plt.plot(n);
mask = np.random.rand(len(data)) < 0.9

Incomplete rows: 98
Loaded: 12890


In [123]:
train = data[mask]
test = data[~mask]
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
print("Test  Total: {} Good: {} Faulty: {} Ratio: {}".format(len(test), len(test[test[:,-1]=='True']), len(test[test[:,-1]=='False']), float(len(test[test[:,-1]=='False']))/len(train)))

Train Total: 11665 Good: 10995 Faulty: 670 Ratio: 0.0574367766824
Test  Total: 1225 Good: 1162 Faulty: 63 Ratio: 0.00540077153879


In [124]:
# faulty = train[train[:,-1]=='False']
# not_faulty = train[train[:,-1]=='True']
# # train = np.concatenate((not_faulty[:len(faulty)*5], faulty))
# samples = np.random.choice(len(not_faulty), 3000, replace=False)
# train = np.concatenate((not_faulty[samples], faulty))
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))

train_data = train[:,2:-1].astype(np.float32)
test_data = test[:,2:-1].astype(np.float32)
"""
Quality_OK is mapped to Faultiness
    'False' -> 1 (Faulty)
    'True'  -> 0 (Good)
"""
train_labels = np.array(train[:,-1]=='False').astype(np.int32)
test_labels = np.array(test[:,-1]=='False').astype(np.int32)

Train Total: 11665 Good: 10995 Faulty: 670 Ratio: 0.0574367766824


In [125]:
from IPython.display import display, HTML
import pandas as pd

def eval_metrics(expected, predicted):
    print("------------------------- EVALUATION -------------------------")
#     print("Accuracy Score: {}".format(metrics.accuracy_score(expected, predicted)))
    print(metrics.classification_report(expected, predicted))
    print("Confusion Matrix:")
#     print(metrics.confusion_matrix(expected, predicted))
    display(pd.DataFrame(metrics.confusion_matrix(expected, predicted)))
    print("Kappa Score: {}".format(metrics.cohen_kappa_score(expected, predicted)))
#     print("Matthews Correlation Coefficient: {}".format(metrics.matthews_corrcoef(expected, predicted)))
    print("--------------------------------------------------------------")
#     plot_confusion_matrix(metrics.confusion_matrix(expected, predicted),classes=['Good','Faulty'],title='Confusion matrix',normalize=True)
#     plt.show()

In [126]:
""" Classification and Regression Trees (CART) """
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# ratio = float(len(train[train[:,-1]=='False']))/len(train)
criteria = ["gini", "entropy"]
clf = DecisionTreeClassifier(criterion=criteria[0], random_state=1, max_depth=100,  class_weight={0:1.0, 1:0.05})
%time clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))
# from sklearn.tree import export_graphviz
# export_graphviz(clf, out_file='tree.dot') 

Wall time: 2.37 s
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10995
          1       1.00      0.99      0.99       670

avg / total       1.00      1.00      1.00     11665

Confusion Matrix:


,0,1
0,10995,0
1,8,662


Kappa Score: 0.993630338482
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1162
          1       0.98      0.94      0.96        63

avg / total       1.00      1.00      1.00      1225

Confusion Matrix:


,0,1
0,1161,1
1,4,59


Kappa Score: 0.957202249939
--------------------------------------------------------------


In [121]:
""" Random Forest """
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100,  class_weight={0:1.0, 1:0.05}, warm_start=False)

%time clf = clf.fit(train_data, train_labels)
eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

Wall time: 6.47 s
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3000
          1       1.00      1.00      1.00       662

avg / total       1.00      1.00      1.00      3662

Confusion Matrix:


,0,1
0,3000,0
1,0,662


Kappa Score: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      1183
          1       0.83      0.94      0.88        71

avg / total       0.99      0.99      0.99      1254

Confusion Matrix:


,0,1
0,1169,14
1,4,67


Kappa Score: 0.87397407122
--------------------------------------------------------------


In [ ]:
import random
from copy import deepcopy
def randomPair(predicted_label):
    index = random.sample(np.where(train_labels==abs(1-predicted_label))[0],1)
    return deepcopy(train_data[index][0]), deepcopy(train_labels[index][0])

for i,t in enumerate(test_data):
    prediction = clf.predict([t])[0]
    pair_data, pair_label = randomPair(prediction)
#     print(test_data[i])
#     print(pair_data)
    clf = clf.fit([test_data[i],pair_data], [prediction,pair_label])

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

In [104]:
""" Bagging Classifier """ 
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics

clf = BaggingClassifier(n_estimators=50, random_state=1)
%time clf = clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

Wall time: 21.5 s
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3000
          1       1.00      1.00      1.00       658

avg / total       1.00      1.00      1.00      3658

Confusion Matrix:


,0,1
0,3000,0
1,0,658


Kappa Score: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      1181
          1       0.85      1.00      0.92        75

avg / total       0.99      0.99      0.99      1256

Confusion Matrix:


,0,1
0,1168,13
1,0,75


Kappa Score: 0.914748757362
--------------------------------------------------------------
